# VideoNetClassification

Collaborators:

- Yahia Ehab
- Mariam Amr
- Mohamed Khaled

## Installation

In [1]:
!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs

^C
ERROR: Operation cancelled by user


In [2]:
import tensorflow as tf

print("Metal available:", tf.config.experimental.list_physical_devices("Metal"))
print("Metal device info:", tf.config.list_physical_devices("Metal"))


Metal available: []
Metal device info: []


## Imports

In [3]:
# @title Import the necessary modules
# TensorFlow and TF-Hub modules.
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3

## Data Loading

In [4]:
import pandas as pd

# Helper functions for the UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()
unverified_context = ssl._create_unverified_context()

def list_ucf_videos():
    """Lists videos available in UCF101 dataset."""
    global _VIDEO_LIST
    if not _VIDEO_LIST:
        index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
        videos = re.findall("(v_[\w_]+\.avi)", index)
        _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)

def fetch_ucf_video(video):
    """Fetches a video and cache into local filesystem."""
    cache_path = os.path.join(_CACHE_DIR, video)
    if not os.path.exists(cache_path):
        urlpath = request.urljoin(UCF_ROOT, video)
        print("Fetching %s => %s" % (urlpath, cache_path))
        data = request.urlopen(urlpath, context=unverified_context).read()
        open(cache_path, "wb").write(data)
    return cache_path

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=30, resize=(224, 224)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if max_frames != 0 and len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames) / 255.0

def to_gif(images):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images, duration=40)
    return embed.embed_file('./animation.gif')

# Define a function to create DataFrame with video paths and labels
def create_dataframe(num_videos=300):
    video_paths = []
    labels = []

    # List all UCF101 videos
    ucf_videos = list_ucf_videos()

    # Randomly select videos
    random_videos = random.sample(ucf_videos, num_videos)

    # Extract labels from video filenames
    for video in random_videos:
        label = video.split('_')[1]
        video_paths.append(fetch_ucf_video(video))
        labels.append(label)

    # Create DataFrame
    df = pd.DataFrame({'video_paths': video_paths, 'labels': labels})
    return df

# Create DataFrame with video paths and labels
df = create_dataframe()

# # Display the DataFrame
# print(df.head())


Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_SumoWrestling_g24_c01.avi => /var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn/T/tmp1xtifctq/v_SumoWrestling_g24_c01.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_Hammering_g01_c03.avi => /var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn/T/tmp1xtifctq/v_Hammering_g01_c03.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_WallPushups_g20_c05.avi => /var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn/T/tmp1xtifctq/v_WallPushups_g20_c05.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_PommelHorse_g17_c01.avi => /var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn/T/tmp1xtifctq/v_PommelHorse_g17_c01.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_HorseRace_g20_c02.avi => /var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn/T/tmp1xtifctq/v_HorseRace_g20_c02.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_BabyCrawling_g05_c02.avi => /var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn/T/tmp

In [5]:
import pandas as pd
# Save the DataFrame to a CSV file
df.to_csv('ucf101_videos_labels.csv', index=False)
#df = pd.read_csv('ucf101_videos_labels.csv')

In [6]:
df.head()

,video_paths,labels
0,/var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn...,SumoWrestling
1,/var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn...,Hammering
2,/var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn...,WallPushups
3,/var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn...,PommelHorse
4,/var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn...,HorseRace


In [7]:
df['video_paths'][0]

'/var/folders/qw/4tzkkgc16ldbw62c8_s7_fkh0000gn/T/tmp1xtifctq/v_SumoWrestling_g24_c01.avi'

### Load Video as GIF

Create `/GIFs` dir

In [8]:
frames_clip = [] # a '2d' array where each element is a group of frames corresponding to one video
for i in range(0, 300):
    # Load the first video from the DataFrame
    video_path = df['video_paths'][i]
    video = load_video(video_path)
    converted_video = np.clip(video*255, 0, 255).astype(np.uint8)
    frames_clip.append(converted_video)
    print("finished video: ", i)

# extract labels from the dataframe
labels = df['labels'].values


finished video:  0
finished video:  1
finished video:  2
finished video:  3
finished video:  4
finished video:  5
finished video:  6
finished video:  7
finished video:  8
finished video:  9
finished video:  10
finished video:  11
finished video:  12
finished video:  13
finished video:  14
finished video:  15
finished video:  16
finished video:  17
finished video:  18
finished video:  19
finished video:  20
finished video:  21
finished video:  22
finished video:  23
finished video:  24
finished video:  25
finished video:  26
finished video:  27
finished video:  28
finished video:  29
finished video:  30
finished video:  31
finished video:  32
finished video:  33
finished video:  34
finished video:  35
finished video:  36
finished video:  37
finished video:  38
finished video:  39
finished video:  40
finished video:  41
finished video:  42
finished video:  43
finished video:  44
finished video:  45
finished video:  46
finished video:  47
finished video:  48
finished video:  49
finished v

In [9]:
num_of_classes = len(df['labels'].unique())

In [10]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

preprocessed = []
for i in range(0, 300):
    preprocessed.append(preprocess_input(frames_clip[i]))
    print("finished video: ", i)

finished video:  0
finished video:  1
finished video:  2
finished video:  3
finished video:  4
finished video:  5
finished video:  6
finished video:  7
finished video:  8
finished video:  9
finished video:  10
finished video:  11
finished video:  12
finished video:  13
finished video:  14
finished video:  15
finished video:  16
finished video:  17
finished video:  18
finished video:  19
finished video:  20
finished video:  21
finished video:  22
finished video:  23
finished video:  24
finished video:  25
finished video:  26
finished video:  27
finished video:  28
finished video:  29
finished video:  30
finished video:  31
finished video:  32
finished video:  33
finished video:  34
finished video:  35
finished video:  36
finished video:  37
finished video:  38
finished video:  39
finished video:  40
finished video:  41
finished video:  42
finished video:  43
finished video:  44
finished video:  45
finished video:  46
finished video:  47
finished video:  48
finished video:  49
finished v

## Preprocessing

- CNN (InceptionV3 Model)
    1. Image Size should be 299*299 (only if we're using the full model)

- RNN
    1. LTSM

### CNN

In [11]:
# Load the InceptionV3 model from TensorFlow Hub
feature_extractor = hub.KerasLayer(
    "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4", trainable=False
)

In [12]:
# Function to extract features using InceptionV3
from tensorflow.keras.applications import InceptionV3

def extract_video_features(video_frames):
    base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
    video_features = []
    for video in video_frames:
        video_features.append(base_model.predict(video))
    return np.array(video_features)

video_features = extract_video_features(preprocessed)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 538ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 535ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 533ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 542ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 528ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 535ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 531ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 556ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 538ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 552ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

In [14]:
split = int(0.8*len(video_features))
train, test = video_features[:split], video_features[split:]
train_labels, test_labels = labels[:split], labels[split:]

In [15]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_onehot = to_categorical(train_labels_encoded)

test_labels_encoded = label_encoder.fit_transform(test_labels)
test_labels_onehot = to_categorical(test_labels_encoded)


In [25]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence

# RNN model creation
rnn_model = Sequential(
    [
        LSTM(128, return_sequences=True, input_shape=(30, 2048)),
        LSTM(64),
        Dense(64, activation="relu"),
        Dropout(0.5),
        Dense(93, activation="softmax"),
    ]
)

rnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

print(train.shape)
print(rnn_model.input_shape)
print(train_labels_onehot.shape)
print(rnn_model.output_shape)

(240, 30, 2048)
(None, 30, 2048)
(240, 93)
(None, 93)


/Users/yahiaehab/University/Projects/MachineLearning/Project_II/env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

rnn_model = Sequential()
rnn_model.add(
    SimpleRNN(50, input_shape=(None, 2048))
)  # Adjust based on your specific model
rnn_model.add(
    Dense(93, activation="softmax")
)  # Adjust the number of neurons to match the number of classes

rnn_model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

In [29]:
rnn_model.fit(x=train, 
              y=train_labels_onehot, 
              epochs=10, 
              batch_size=32,
            validation_split=0.2)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.0065 - loss: 4.6930 - val_accuracy: 0.0417 - val_loss: 4.5417
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0783 - loss: 4.3386 - val_accuracy: 0.0625 - val_loss: 4.5638
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2247 - loss: 3.9587 - val_accuracy: 0.1250 - val_loss: 4.4385
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2905 - loss: 3.6623 - val_accuracy: 0.0833 - val_loss: 4.3915
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4026 - loss: 3.3072 - val_accuracy: 0.1875 - val_loss: 4.3094
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4531 - loss: 3.1460 - val_accuracy: 0.1875 - val_loss: 4.3208
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6479 - loss: 2.8519 - val_accuracy: 0.1875 - val_loss: 4.2412
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7224 - loss: 2.6425 - val_accuracy: 0.1875 - val_loss: 4.2587


In [34]:
rnn_model.evaluate(x=test, y=test_labels_onehot)


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 49), output.shape=(None, 93)